# WordCount in Spark (work in-progress)

The WordCount problem is to read a text and then count the number of occurrence of each word. In the MapReduce paradigm, in the `map` phase, each mapper takes a line as an input and tokenizes it. It then spits out a key-value pair with the key as the word and value as 1 for each key. In the `reduce` step, each reducer sums the count of each word and emits a key value pair with each unique word as a key and the total number of occurrence of that word as a value.  

In this notebook, we will read a text file with two sentences stored in the local file system. This file can be found in the `data` directory of this repository. 

In [14]:
text_RDD = sc.textFile('./data/textfile_1.txt')

In [2]:
type(text_RDD)

pyspark.rdd.RDD

### The `map` step 
In this step, we'll be doing the following two operations:
1. Split the sentences into tokens i.e. words. This is taken care of by the `split_word` function in the snippted below.
2. Create key-value pairs where the key is a word and the value is a 1. This is taken care of by the `create_kv_pair` function in snippet below.

`flatMap()` applies a function (in this case `split_word`) to each chunk of the RDD.

In [12]:
def split_word(line):
    return line.split()

def create_kv_pair(word):
    return (word,1)

pairs_RDD = text_RDD.flatMap(split_word).map(create_kv_pair)
print type(text_RDD.flatMap(split_word))
print "Type returned by split_word", text_RDD.flatMap(split_word)
print type(pairs_RDD)
print pairs_RDD.collect()
print type(pairs_RDD.collect())

<class 'pyspark.rdd.PipelinedRDD'>
Type returned by split_word PythonRDD[24] at RDD at PythonRDD.scala:48
<class 'pyspark.rdd.PipelinedRDD'>
[(u'A', 1), (u'quick', 1), (u'brown', 1), (u'fox', 1), (u'jumped', 1), (u'over', 1), (u'a', 1), (u'lazy', 1), (u'dog.', 1), (u'A', 1), (u'quick', 1), (u'brown', 1), (u'dog', 1), (u'jumped', 1), (u'over', 1), (u'a', 1), (u'lazy', 1), (u'fox.', 1)]
<type 'list'>


### The `reduce` step
In this step, we want to count the number of occurrences of each word. So, what we need is an aggregation of counts by key. User defined function `sum_counts` and the in-built function `reduceByKey` helps us achieve this objective. 

In [4]:
def sum_counts(a, b):
    return a + b
wordcounts_RDD = pairs_RDD.reduceByKey(sum_counts)
wordcounts_RDD.collect()

[(u'A', 2),
 (u'dog.', 1),
 (u'lazy', 2),
 (u'over', 2),
 (u'fox', 1),
 (u'a', 2),
 (u'quick', 2),
 (u'brown', 2),
 (u'dog', 1),
 (u'jumped', 2),
 (u'fox.', 1)]

### Transformations

#### `flatMap`

In [5]:
words_RDD = text_RDD.flatMap(split_word)
words_RDD.collect()

[u'A',
 u'quick',
 u'brown',
 u'fox',
 u'jumped',
 u'over',
 u'a',
 u'lazy',
 u'dog.',
 u'A',
 u'quick',
 u'brown',
 u'dog',
 u'jumped',
 u'over',
 u'a',
 u'lazy',
 u'fox.']

#### `filter`

In [6]:
def starts_with_a(word):
    return word.lower().startswith("a")
words_RDD.filter(starts_with_a).collect()

[u'A', u'a', u'A', u'a']

#### `groupByKey`

In [7]:
pairs_RDD.groupByKey().collect()

[(u'A', <pyspark.resultiterable.ResultIterable at 0x7f4b294a3890>),
 (u'dog.', <pyspark.resultiterable.ResultIterable at 0x7f4b294a36d0>),
 (u'lazy', <pyspark.resultiterable.ResultIterable at 0x7f4b294a3bd0>),
 (u'over', <pyspark.resultiterable.ResultIterable at 0x7f4b294a3c10>),
 (u'fox', <pyspark.resultiterable.ResultIterable at 0x7f4b294a3cd0>),
 (u'a', <pyspark.resultiterable.ResultIterable at 0x7f4b294a3350>),
 (u'quick', <pyspark.resultiterable.ResultIterable at 0x7f4b294a3450>),
 (u'brown', <pyspark.resultiterable.ResultIterable at 0x7f4b291e2710>),
 (u'dog', <pyspark.resultiterable.ResultIterable at 0x7f4b294797d0>),
 (u'jumped', <pyspark.resultiterable.ResultIterable at 0x7f4b29479390>),
 (u'fox.', <pyspark.resultiterable.ResultIterable at 0x7f4b29479690>)]

In [8]:
for k,v in pairs_RDD.groupByKey().collect():
    print "Key:", k, ", Values:", list(v)

Key: A , Values: [1, 1]
Key: dog. , Values: [1]
Key: lazy , Values: [1, 1]
Key: over , Values: [1, 1]
Key: fox , Values: [1]
Key: a , Values: [1, 1]
Key: quick , Values: [1, 1]
Key: brown , Values: [1, 1]
Key: dog , Values: [1]
Key: jumped , Values: [1, 1]
Key: fox. , Values: [1]
